# Machine learning
## Test de différents algorithmes de machine learning pour de la classification des espèces d'iris

Dans ce travail, nous aurons besoin d'une librairie R contenant des centaines d'algorithmes de machine learnig (ces algorithmes sont déjà codés, nous allons "juste" les utilser !). Cette librairie s'appelle "caret". Les deux premières librairies contiennent des outils de visualisation des données.

In [ ]:
library("ggplot2") # graphiques classiques

In [ ]:
library("psych") # pour la visualisation des données 

In [ ]:
library("caret") # machine learning

Dans ce NoteBook, on va travailler avec des données de fleurs - des iris - déjà implémentées sous R. Ce jeu de données est très classique pour la découverte du machine learning et contient 150 observations de d'iris. Pour charger ce jeu de données, excécutez les commandes suivantes : 

In [ ]:
data(iris)

Le jeu de données comprend 50 échantillons de chacune des trois espèces d'iris (Iris setosa, Iris virginica et Iris versicolor).  Quatre caractéristiques ont été mesurées à partir de chaque échantillon : la longueur et la largeur des sépales et des pétales, en centimètres. 

<img src="iris2.jpg" style="width:700px;height:300px"/>

Regardons ce jeu de données : 

In [ ]:
iris

## 1) Explorons les données

Taille de notre jeu de données :

In [ ]:
dim(iris)

### a) matrices de corrélation 

In [ ]:
pairs.panels(
       iris[,1:4], # Our data.
       scale = TRUE, # Changes size of correlation value lables based on strength.
       hist.col = 'grey85', # Histogram color.  
       bg = c("mediumseagreen","orange2","mediumpurple1")[iris$Species], # Colors of the Species levels.
       pch = 21, # The plot characters shape and size.
       main = 'Correlation matrix of Iris data')  # Title. 

### b) Boxplot de la largeur des sépales (Sepal.Width)

In [ ]:
ggplot(
       # (1) Set data; (2) Specify X and Y variables; (3) 'fill' color separates our Species levels.
       data = iris, mapping = aes(x = Species, y = Sepal.Width, fill = Species)) +
       geom_boxplot() +  # Specifies that we want a box plot. 
       scale_fill_brewer(palette = 'Dark2') +  # Change color of box plots. 
       theme_light() +  # Set light theme. 
       labs(title = 'Box plot of sepal width for each species', 
            x = 'Species', y = 'Sepal width')  # Assign a title, axis names.

### c) Boxplot de la longueur des sépales (Sepal.Length)

In [ ]:
ggplot(data = iris, mapping = aes(x = Species, y = Sepal.Length, fill = Species)) +
       geom_boxplot() + 
       scale_fill_brewer(palette = 'Dark2') + 
       theme_light() +
       labs(title = 'Box plot of sepal length for each species', 
            x = 'Species', y = 'Sepal length')

### d) Boxplots pour les pétales

Créer deux boxplots : l'un pour la largeur des pétales et l'autre pour la longueur des pétales, en vous aidant du code ci-dessus.

### Largeur des pétales :

### Longueur des pétales :

## 2) Création du jeu de données "entrainement" et "test" 

Avant d'utiliser les algorithmes de machine learning, il va falloir séparer notre jeu de données en deux : l'un pour de l'entrainement (l'algorithme de machine learning va s'appuyer sur ce jeu de données pour apprendre à faire des prévisions sur nos données) et l'autre pour tester, c'est à dire évaluer si l'algorithme arrive à prévoir correctement. On peut faire des statistiques sur le jeu de test pour savoir si le modèle est correct ou pas !

On va créer un jeu d'entrainement à l'aide du paquet CARET qui va représenter 70% du jeu de données iris. Les données sont choisies aléatoirement.

In [ ]:
train_index <- createDataPartition(y = iris$Species,  # y = our dependent variable.
                                   p = 0.7,  # Specifies split into 70% & 30%.
                                   list = FALSE,  # Sets results to matrix form. 
                                   times = 1)  # Sets number of partitions to create to 1. 

Jeu de données d'entrainement :

In [ ]:
train_data <- iris[train_index,] 

Jeu de données de test : 

In [ ]:
test_data <- iris[-train_index,]

Quelle est la dimension du jeu d'entrainement et de test ? 

In [ ]:
dim(train_data)
dim(test_data)

Explorer les deux jeux de données :

## 3) Les différents algorithmes de machine learning

L'objectif du machine learning dans notre problème est de prévoir l'espèce d'iris en fonction des caractéristiques (longueur et largeur) des pétales et sépales.


Nous allons utiliser différents algorithmes classiques de machines learning :
- un méthode linéaire (LDA) 
- une méthode avec un arbre de décision (CART) 
- une méthode du plus proche voisin (kNN) 
- Support Vector Machine (SVM) 

Tous ces algrorithmes font parties des méthodes par apprentissage supervisé : le système est guidé dans son apprentissage en le nourissant d'exemples (par opposition aux méthodes non supervisés et par renforcement).

Deux grands types de méthodes existent pour mener à bien un apprentissage supervisé :
- la régression
- la classification

<img src="ml.PNG" style="width:600px;height:300px"/>

LDA est une méthode par régression alors que CART et kNN sont des méthodes de classification. SVM, quant à lui, mélange des régressions et des classifications.

La précsion du jeu d'entrainement est évaluée en machine learnig. C'est un premier indicateur nous permettant de savoir si le modèle obtenu par machine learning semble robuste ou pas.

In [ ]:
control <- trainControl(method="cv", number=10)
metric <- "Accuracy"

## a) Méthode linéaire 

### Entrainement 

Création du modèle de prévision avec la méthode linéaire (LDA).

In [ ]:
lda_model <- train(Species~., data=train_data, method="lda", metric=metric, trControl=control)

Evaluons la précision de notre modèle à l'aide d'une matrice de confusion :

In [ ]:
confusionMatrix(lda_model)

On peut aussi regarder l'importance de chaque variable dans le choix de la classification de l'espèce d'iris.

In [ ]:
# Create object of importance of our variables 
dt_importance <- varImp(lda_model)

# Create plot of importance of variables
ggplot(data = dt_importance, mapping = aes(x = dt_importance[,1])) + # Data & mapping
  geom_boxplot() + # Create box plot
  labs(title = "Variable importance: Decision tree model") + # Title
  theme_light() # Theme

Quelles conclusions en tirez-vous ? 

### Prévision : utilisation du modèle sur le jeu de données test

In [ ]:
prediction_lda <- predict(lda_model, test_data)

Regardons ce que nous obtenons :

In [ ]:
prediction_lda

Déterminons la proportion de prévisions justes à l'aide de matrices de confusion :

In [ ]:
confusionMatrix(prediction_lda, test_data$Species)

## b) Arbre de décision

### Entrainement 

Création du modèle de prévision à l'aide d'un arbre de décision (CART). Sous R, ce modèle se nomme "rpart".

In [ ]:
cart_model <- train(Species~., data=train_data, method="rpart", metric=metric, trControl=control)

Evaluons la précision de notre modèle à l'aide d'une matrice de confusion :

In [ ]:
confusionMatrix(cart_model)

On peut aussi regarder l'importance de chaque variable dans le choix de la classification de l'espèce d'iris.

In [ ]:
# Create object of importance of our variables 
dt_importance <- varImp(cart_model)

# Create plot of importance of variables
ggplot(data = dt_importance, mapping = aes(x = dt_importance[,1])) + # Data & mapping
  geom_boxplot() + # Create box plot
  labs(title = "Variable importance: Decision tree model") + # Title
  theme_light() # Theme

Quelles conclusions en tirez-vous ? 

### Prévision : utilisation du modèle sur le jeu de données test

In [ ]:
prediction_cart <- predict(cart_model, test_data)

Déterminons la proportion de prévisions justes à l'aide de matrices de confusion :

In [ ]:
confusionMatrix(prediction_cart, test_data$Species)

## c) Méthode du plus proche voisin 

### Entrainement 

Création du modèle de prévision à l'aide de la méthode du plus proche voisin (kNN).

In [ ]:
knn_model <- train(Species~., data=train_data, method="knn", metric=metric, trControl=control)

Evaluons la précision de notre modèle à l'aide d'une matrice de confusion :

In [ ]:
confusionMatrix(knn_model)

On peut aussi regarder l'importance de chaque variable dans le choix de la classification de l'espèce d'iris.

In [ ]:
# Create object of importance of our variables 
dt_importance <- varImp(knn_model)

# Create plot of importance of variables
ggplot(data = dt_importance, mapping = aes(x = dt_importance[,1])) + # Data & mapping
  geom_boxplot() + # Create box plot
  labs(title = "Variable importance: Decision tree model") + # Title
  theme_light() # Theme

Quelles conclusions en tirez-vous ? 

### Prévision : utilisation du modèle sur le jeu de données test

In [ ]:
prediction_knn <- predict(knn_model, test_data)

Déterminons la proportion de prévisions justes à l'aide de matrices de confusion :

In [ ]:
confusionMatrix(prediction_cart, test_data$Species)

## d) Support Vector Machine

### Entrainement 

Création du modèle de prévision à l'aide du support vector machine. Sur R, cette méthode se nomme "svmRadial".

In [ ]:
svm_model <- train(Species~., data=train_data, method="svmRadial", metric=metric, trControl=control)

Evaluons la précision de notre modèle à l'aide d'une matrice de confusion :

In [ ]:
confusionMatrix(svm_model)

On peut aussi regarder l'importance de chaque variable dans le choix de la classification de l'espèce d'iris.

In [ ]:
# Create object of importance of our variables 
dt_importance <- varImp(svm_model)

# Create plot of importance of variables
ggplot(data = dt_importance, mapping = aes(x = dt_importance[,1])) + # Data & mapping
  geom_boxplot() + # Create box plot
  labs(title = "Variable importance: Decision tree model") + # Title
  theme_light() # Theme

Quelles conclusions en tirez-vous ? 

### Prévision : utilisation du modèle sur le jeu de données test

In [ ]:
prediction_svm <- predict(svm_model, test_data)

Déterminons la proportion de prévisions justes à l'aide de matrices de confusion :

In [ ]:
confusionMatrix(prediction_svm, test_data$Species)

# 3) Comparaison des modèles

Comparons les 4 modèles entre eux pour savoir celui qui est le plus précis. Listons tous les modèles au sein d'une même variable :

In [ ]:
results <- resamples(list(lda=lda_model, cart=cart_model, knn=knn_model, svm=svm_model))
summary(results)

On peut représenter ces données à l'aide d'un graphique :

In [ ]:
dotplot(results)

Quel est le modèle le plus précis ?